## Web scraping to enrich our CSV ##

In [ ]:
import pandas as pd  
import numpy as np
import re
import pickle
from tqdm import tqdm


from selenium import webdriver     # I import everything I need to do web scrapping with selenium
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException



# Ignoring the warnings.
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# I activate all these options for doing the web scrapping

opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
#para ocultarme como robot
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized') 
opciones.add_argument('user.data-dir=selenium')
opciones.add_argument('--incognito

In [194]:
movies = pd.read_csv('/Users/davidusuariocasa/Desktop/Projects/ETL_project/CSVs/Oscars_cleaned.csv') # I open the csv with the CSV I want to iterate from 

## Loop to get each film's 'score' and 'duration' from IMDB ##


In [279]:
driver = webdriver.Firefox()

driver.get("https://www.imdb.com/") # This is the web page we get the info from
driver.implicitly_wait(2)

for index, (value, year) in enumerate(zip(movies['film'], movies['year_film'])): # We iterate through each film (row), and we want to write its title and year
    try:
        
        driver.find_element("css selector", "#suggestion-search").send_keys(f'{value} {str(year)}') # In IMDB's search box, we write each film and its year
        time.sleep(2)

        try:
            
            driver.find_element("css selector", "#react-autowhatever-navSuggestionSearch--item-0 > a:nth-child(1) > div:nth-child(2)").click() # Try to click on the first option that our seach shows
        except:
            movies.loc[index, 'score'] = np.nan # If our movie doesn't have an IMDB profile, we fill our column 'score' and 'duration' with null values, and clear our search box
            movies.loc[index, 'duration'] = np.nan
            driver.find_element("css selector", "#suggestion-search").clear()

            continue  # Continue to the next iteration

        try:
            duration = driver.find_element('xpath', '/html/body/div[2]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/ul/li[3]').text
            movies.loc[index, 'duration'] = duration # If the film has a duration written, take it and assign it to our 'duration' column
        except:
            movies.loc[index, 'duration'] = np.nan # If not, assign a null value to the 'duration' column
            continue  # Continue to the next iteration

        try:
            score = driver.find_element('xpath', '/html/body/div[2]/main/div/section[1]/section/div[3]/section/section/div[2]/div[2]/div/div[1]/a/span/div/div[2]/div[1]').text
            movies.loc[index, 'score'] = score # If the film has a score written, take it and assign it to our 'score' column
        except:
            movies.loc[index, 'score'] = np.nan # If not, assign a null value to the 'score' column

    except TimeoutException:
        continue  # Go to the next iteration, if it shows an error of Timeout Exception 

driver.quit() # At the end of the for loop, close the Firefox navegator 



## First iteration ##

In [212]:
'''First time we iterated through our 'movie' df, an error appeared at index 260, so we take the first 260 
indexes (because they took the data that we wanted), and store them in a new variable called 'movies_sub1''''

movies_sub1 = movies.iloc[:260] # This is our first subdf, with the data we wanted before the error


In [221]:
movies_sub2 = movies.iloc[260:]

In [222]:
movies_sub2 = movies_sub2.reset_index()
movies_sub2 = movies_sub2.drop('index', axis=1)
movies_sub2

,year_film,year_ceremony,ceremony,category,name,film,duration,score
0,1943,1944,16,SOUND RECORDING,"RKO Radio Studio Sound Department, Stephen Dun...",This Land Is Mine,NaN,NaN
1,1943,1944,16,SPECIAL EFFECTS,Photographic Effects by Fred Sersen; Sound Eff...,Crash Dive,NaN,NaN
2,1943,1944,16,WRITING (Original Motion Picture Story),William Saroyan,The Human Comedy,NaN,NaN
3,1943,1944,16,WRITING (Original Screenplay),Norman Krasna,Princess O'Rourke,NaN,NaN
4,1943,1944,16,WRITING (Screenplay),"Julius J. Epstein, Philip G. Epstein, Howard Koch",Casablanca,NaN,NaN
...,...,...,...,...,...,...,...,...
1884,2022,2023,95,SOUND,"Mark Weingarten, James H. Mather, Al Nelson, C...",Top Gun: Maverick,NaN,NaN
1885,2022,2023,95,VISUAL EFFECTS,"Joe Letteri, Richard Baneham, Eric Saindon and...",Avatar: The Way of Water,NaN,NaN
1886,2022,2023,95,WRITING (Adapted Screenplay),Screenplay by Sarah Polley,Women Talking,NaN,NaN
1887,2022,2023,95,WRITING (Original Screenplay),Written by Daniel Kwan & Daniel Scheinert,Everything Everywhere All at Once,NaN,NaN


In [234]:
movies_sub2 = movies_sub2.iloc[:419]

In [235]:
movies_sub2 # FINAL DF 2 

,year_film,year_ceremony,ceremony,category,name,film,duration,score
0,1943,1944,16,SOUND RECORDING,"RKO Radio Studio Sound Department, Stephen Dun...",This Land Is Mine,1h 43min,"7,5\n/10"
1,1943,1944,16,SPECIAL EFFECTS,Photographic Effects by Fred Sersen; Sound Eff...,Crash Dive,1h 46min,"6,4\n/10"
2,1943,1944,16,WRITING (Original Motion Picture Story),William Saroyan,The Human Comedy,1h 57min,"7,0\n/10"
3,1943,1944,16,WRITING (Original Screenplay),Norman Krasna,Princess O'Rourke,1h 34min,"6,7\n/10"
4,1943,1944,16,WRITING (Screenplay),"Julius J. Epstein, Philip G. Epstein, Howard Koch",Casablanca,NaN,NaN
...,...,...,...,...,...,...,...,...
414,1960,1961,33,DOCUMENTARY (Feature),"Larry Lansburgh, Producer",The Horse with the Flying Tail,48min,"6,1\n/10"
415,1960,1961,33,DOCUMENTARY (Short Subject),"James Hill, Producer",Giuseppina,NaN,NaN
416,1960,1961,33,FILM EDITING,Daniel Mandell,The Apartment,2h 5min,"8,3\n/10"
417,1960,1961,33,FOREIGN LANGUAGE FILM,Sweden,The Virgin Spring,1h 29min,"8,1\n/10"


In [262]:
movies_sub3 = movies.iloc[679:]
movies_sub3 = movies_sub3.reset_index()
movies_sub3 = movies_sub3.drop('index', axis=1)
movies_sub3

,year_film,year_ceremony,ceremony,category,name,film,duration,score
0,1960,1961,33,MUSIC (Scoring of a Musical Picture),"Morris Stoloff, Harry Sukman",Song without End (The Story of Franz Liszt),NaN,NaN
1,1960,1961,33,MUSIC (Song),Music and Lyrics by Manos Hadjidakis,Never on Sunday,NaN,NaN
2,1960,1961,33,BEST MOTION PICTURE,"Billy Wilder, Producer",The Apartment,NaN,NaN
3,1960,1961,33,SHORT SUBJECT (Cartoon),"William L. Snyder, Producer",Munro,NaN,NaN
4,1960,1961,33,SHORT SUBJECT (Live Action),"Ezra R. Baker, Producer",Day of the Painter,NaN,NaN
...,...,...,...,...,...,...,...,...
1465,2022,2023,95,SOUND,"Mark Weingarten, James H. Mather, Al Nelson, C...",Top Gun: Maverick,NaN,NaN
1466,2022,2023,95,VISUAL EFFECTS,"Joe Letteri, Richard Baneham, Eric Saindon and...",Avatar: The Way of Water,NaN,NaN
1467,2022,2023,95,WRITING (Adapted Screenplay),Screenplay by Sarah Polley,Women Talking,NaN,NaN
1468,2022,2023,95,WRITING (Original Screenplay),Written by Daniel Kwan & Daniel Scheinert,Everything Everywhere All at Once,NaN,NaN


In [272]:
movies_sub3.head(723)

,year_film,year_ceremony,ceremony,category,name,film,duration,score
0,1960,1961,33,MUSIC (Scoring of a Musical Picture),"Morris Stoloff, Harry Sukman",Song without End (The Story of Franz Liszt),NaN,NaN
1,1960,1961,33,MUSIC (Song),Music and Lyrics by Manos Hadjidakis,Never on Sunday,1h 31min,"7,3\n/10"
2,1960,1961,33,BEST MOTION PICTURE,"Billy Wilder, Producer",The Apartment,2h 5min,"8,3\n/10"
3,1960,1961,33,SHORT SUBJECT (Cartoon),"William L. Snyder, Producer",Munro,1h 37min,"5,5\n/10"
4,1960,1961,33,SHORT SUBJECT (Live Action),"Ezra R. Baker, Producer",Day of the Painter,NaN,NaN
...,...,...,...,...,...,...,...,...
718,1991,1992,64,FILM EDITING,"Joe Hutshing, Pietro Scalia",JFK,3h 9min,"8,0\n/10"
719,1991,1992,64,FOREIGN LANGUAGE FILM,Italy,Mediterraneo,NaN,NaN
720,1991,1992,64,MAKEUP,"Stan Winston, Jeff Dawn",Terminator 2: Judgment Day,NaN,NaN
721,1991,1992,64,MUSIC (Original Score),Alan Menken,Beauty and the Beast,NaN,NaN


In [273]:
movies_sub3 = movies_sub3.iloc[:719]

In [274]:
movies_sub3 # FINAL DF3

,year_film,year_ceremony,ceremony,category,name,film,duration,score
0,1960,1961,33,MUSIC (Scoring of a Musical Picture),"Morris Stoloff, Harry Sukman",Song without End (The Story of Franz Liszt),NaN,NaN
1,1960,1961,33,MUSIC (Song),Music and Lyrics by Manos Hadjidakis,Never on Sunday,1h 31min,"7,3\n/10"
2,1960,1961,33,BEST MOTION PICTURE,"Billy Wilder, Producer",The Apartment,2h 5min,"8,3\n/10"
3,1960,1961,33,SHORT SUBJECT (Cartoon),"William L. Snyder, Producer",Munro,1h 37min,"5,5\n/10"
4,1960,1961,33,SHORT SUBJECT (Live Action),"Ezra R. Baker, Producer",Day of the Painter,NaN,NaN
...,...,...,...,...,...,...,...,...
714,1991,1992,64,COSTUME DESIGN,Albert Wolsky,Bugsy,2h 16min,"6,8\n/10"
715,1991,1992,64,DIRECTING,Jonathan Demme,The Silence of the Lambs,1h 58min,"8,6\n/10"
716,1991,1992,64,DOCUMENTARY (Feature),"Allie Light and Irving Saraf, Producers",In the Shadow of the Stars,1h 33min,"6,5\n/10"
717,1991,1992,64,DOCUMENTARY (Short Subject),"Debra Chasnoff, Producer","Deadly Deception: General Electric, Nuclear We...",NaN,NaN


In [277]:
movies_sub4 = movies.iloc[1398:]
movies_sub4 = movies_sub4.reset_index()
movies_sub4 = movies_sub4.drop('index', axis=1)
movies_sub4


,year_film,year_ceremony,ceremony,category,name,film,duration,score
0,1991,1992,64,FOREIGN LANGUAGE FILM,Italy,Mediterraneo,NaN,NaN
1,1991,1992,64,MAKEUP,"Stan Winston, Jeff Dawn",Terminator 2: Judgment Day,NaN,NaN
2,1991,1992,64,MUSIC (Original Score),Alan Menken,Beauty and the Beast,NaN,NaN
3,1991,1992,64,MUSIC (Original Song),Music by Alan Menken; Lyric by Howard Ashman,Beauty and the Beast,NaN,NaN
4,1991,1992,64,BEST PICTURE,"Edward Saxon, Kenneth Utt and Ron Bozman, Prod...",The Silence of the Lambs,NaN,NaN
...,...,...,...,...,...,...,...,...
746,2022,2023,95,SOUND,"Mark Weingarten, James H. Mather, Al Nelson, C...",Top Gun: Maverick,NaN,NaN
747,2022,2023,95,VISUAL EFFECTS,"Joe Letteri, Richard Baneham, Eric Saindon and...",Avatar: The Way of Water,NaN,NaN
748,2022,2023,95,WRITING (Adapted Screenplay),Screenplay by Sarah Polley,Women Talking,NaN,NaN
749,2022,2023,95,WRITING (Original Screenplay),Written by Daniel Kwan & Daniel Scheinert,Everything Everywhere All at Once,NaN,NaN


In [282]:
movies_sub1

,year_film,year_ceremony,ceremony,category,name,film,duration,score
0,1927,1928,1,ACTOR,Emil Jannings,The Last Command,1h 28min,"8,0\n/10"
1,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,NaN,NaN
2,1927,1928,1,ART DIRECTION,William Cameron Menzies,The Dove;,1h 30min,"6,5\n/10"
3,1927,1928,1,CINEMATOGRAPHY,Charles Rosher,Sunrise,1h 34min,"8,1\n/10"
4,1927,1928,1,CINEMATOGRAPHY,Karl Struss,Sunrise,1h 34min,"8,1\n/10"
...,...,...,...,...,...,...,...,...
255,1943,1944,16,MUSIC (Song),Music by Harry Warren; Lyrics by Mack Gordon,"Hello, Frisco, Hello",1h 39min,"6,5\n/10"
256,1943,1944,16,OUTSTANDING MOTION PICTURE,Warner Bros.,Casablanca,NaN,NaN
257,1943,1944,16,SHORT SUBJECT (Cartoon),"Frederick Quimby, Producer",Yankee Doodle Mouse,8min,"7,7\n/10"
258,1943,1944,16,SHORT SUBJECT (One-reel),"Grantland Rice, Producer",Amphibious Fighters,NaN,NaN


In [283]:
movies_sub2

,year_film,year_ceremony,ceremony,category,name,film,duration,score
0,1943,1944,16,SOUND RECORDING,"RKO Radio Studio Sound Department, Stephen Dun...",This Land Is Mine,1h 43min,"7,5\n/10"
1,1943,1944,16,SPECIAL EFFECTS,Photographic Effects by Fred Sersen; Sound Eff...,Crash Dive,1h 46min,"6,4\n/10"
2,1943,1944,16,WRITING (Original Motion Picture Story),William Saroyan,The Human Comedy,1h 57min,"7,0\n/10"
3,1943,1944,16,WRITING (Original Screenplay),Norman Krasna,Princess O'Rourke,1h 34min,"6,7\n/10"
4,1943,1944,16,WRITING (Screenplay),"Julius J. Epstein, Philip G. Epstein, Howard Koch",Casablanca,NaN,NaN
...,...,...,...,...,...,...,...,...
414,1960,1961,33,DOCUMENTARY (Feature),"Larry Lansburgh, Producer",The Horse with the Flying Tail,48min,"6,1\n/10"
415,1960,1961,33,DOCUMENTARY (Short Subject),"James Hill, Producer",Giuseppina,NaN,NaN
416,1960,1961,33,FILM EDITING,Daniel Mandell,The Apartment,2h 5min,"8,3\n/10"
417,1960,1961,33,FOREIGN LANGUAGE FILM,Sweden,The Virgin Spring,1h 29min,"8,1\n/10"


In [284]:
movies_sub3

,year_film,year_ceremony,ceremony,category,name,film,duration,score
0,1960,1961,33,MUSIC (Scoring of a Musical Picture),"Morris Stoloff, Harry Sukman",Song without End (The Story of Franz Liszt),NaN,NaN
1,1960,1961,33,MUSIC (Song),Music and Lyrics by Manos Hadjidakis,Never on Sunday,1h 31min,"7,3\n/10"
2,1960,1961,33,BEST MOTION PICTURE,"Billy Wilder, Producer",The Apartment,2h 5min,"8,3\n/10"
3,1960,1961,33,SHORT SUBJECT (Cartoon),"William L. Snyder, Producer",Munro,1h 37min,"5,5\n/10"
4,1960,1961,33,SHORT SUBJECT (Live Action),"Ezra R. Baker, Producer",Day of the Painter,NaN,NaN
...,...,...,...,...,...,...,...,...
714,1991,1992,64,COSTUME DESIGN,Albert Wolsky,Bugsy,2h 16min,"6,8\n/10"
715,1991,1992,64,DIRECTING,Jonathan Demme,The Silence of the Lambs,1h 58min,"8,6\n/10"
716,1991,1992,64,DOCUMENTARY (Feature),"Allie Light and Irving Saraf, Producers",In the Shadow of the Stars,1h 33min,"6,5\n/10"
717,1991,1992,64,DOCUMENTARY (Short Subject),"Debra Chasnoff, Producer","Deadly Deception: General Electric, Nuclear We...",NaN,NaN


In [285]:
movies_sub4

,year_film,year_ceremony,ceremony,category,name,film,duration,score
0,1991,1992,64,FOREIGN LANGUAGE FILM,Italy,Mediterraneo,1h 36min,"7,4\n/10"
1,1991,1992,64,MAKEUP,"Stan Winston, Jeff Dawn",Terminator 2: Judgment Day,2h 17min,"8,6\n/10"
2,1991,1992,64,MUSIC (Original Score),Alan Menken,Beauty and the Beast,1h 24min,"8,0\n/10"
3,1991,1992,64,MUSIC (Original Song),Music by Alan Menken; Lyric by Howard Ashman,Beauty and the Beast,1h 24min,"8,0\n/10"
4,1991,1992,64,BEST PICTURE,"Edward Saxon, Kenneth Utt and Ron Bozman, Prod...",The Silence of the Lambs,1h 58min,"8,6\n/10"
...,...,...,...,...,...,...,...,...
746,2022,2023,95,SOUND,"Mark Weingarten, James H. Mather, Al Nelson, C...",Top Gun: Maverick,2h 10min,"8,3\n/10"
747,2022,2023,95,VISUAL EFFECTS,"Joe Letteri, Richard Baneham, Eric Saindon and...",Avatar: The Way of Water,3h 12min,"7,6\n/10"
748,2022,2023,95,WRITING (Adapted Screenplay),Screenplay by Sarah Polley,Women Talking,NaN,NaN
749,2022,2023,95,WRITING (Original Screenplay),Written by Daniel Kwan & Daniel Scheinert,Everything Everywhere All at Once,NaN,NaN


In [286]:
df_final = pd.concat([movies_sub1, movies_sub2, movies_sub3, movies_sub4], ignore_index=True)


In [292]:
df_final

,year_film,year_ceremony,ceremony,category,name,film,duration,score
0,1927,1928,1,ACTOR,Emil Jannings,The Last Command,1h 28min,"8,0\n/10"
1,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,NaN,NaN
2,1927,1928,1,ART DIRECTION,William Cameron Menzies,The Dove;,1h 30min,"6,5\n/10"
3,1927,1928,1,CINEMATOGRAPHY,Charles Rosher,Sunrise,1h 34min,"8,1\n/10"
4,1927,1928,1,CINEMATOGRAPHY,Karl Struss,Sunrise,1h 34min,"8,1\n/10"
...,...,...,...,...,...,...,...,...
2144,2022,2023,95,SOUND,"Mark Weingarten, James H. Mather, Al Nelson, C...",Top Gun: Maverick,2h 10min,"8,3\n/10"
2145,2022,2023,95,VISUAL EFFECTS,"Joe Letteri, Richard Baneham, Eric Saindon and...",Avatar: The Way of Water,3h 12min,"7,6\n/10"
2146,2022,2023,95,WRITING (Adapted Screenplay),Screenplay by Sarah Polley,Women Talking,NaN,NaN
2147,2022,2023,95,WRITING (Original Screenplay),Written by Daniel Kwan & Daniel Scheinert,Everything Everywhere All at Once,NaN,NaN


In [293]:
df_final.to_csv('scrapped_csv', index=False)
